<a href="https://colab.research.google.com/github/shubhamksingh1/TextClassification/blob/main/ContentBasedTFIDFWord2VecGlove/Train/Recommendation_Content_Preference_Based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading Cleaned File and taking a peek at it

## Import Libraries

In [ ]:
import pandas as pd
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
# Import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
import pickle

## Reading Files

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Content_Recommendation/recommendation_v3.csv")

## Checking out CSV

In [ ]:
df.describe(include='all')

,asset_id,asset_description,asset_title,custom_stop_desc,custom_stop_title,asset_description_cleaned,asset_title_cleaned
count,82.000000,82,82,82,82,82,82
unique,NaN,82,82,78,35,82,82
top,NaN,how has the retail industry s idea of superior...,retail understanding your customer performance...,"['a', 'an', 'on', 'at', 'is', 's', 'by', 'to',...",[],globally celebrated advisor executive top comp...,case study artificial intelligence big part fu...
freq,NaN,1,1,3,29,1,1
mean,42.085366,NaN,NaN,NaN,NaN,NaN,NaN
std,24.574001,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,21.250000,NaN,NaN,NaN,NaN,NaN,NaN
50%,41.500000,NaN,NaN,NaN,NaN,NaN,NaN
75%,62.750000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(82, 7)

In [ ]:
df.head()

,asset_id,asset_description,asset_title,custom_stop_desc,custom_stop_title,asset_description_cleaned,asset_title_cleaned
0,1,peter drucker the legendary management scholar...,the ascent of the smarter faster lighter enter...,"['3a', 'a', 'an', 'on', 'at', 'is', 's', 'do',...",['of'],peter drucker legendary management scholar con...,ascent smarter faster lighter enterprise
1,2,the software development community has been us...,agile s day has truly arrived,"['a', 'u', 'we', 'k', 'i', 'on', 'at', 'is', '...",['s'],software development community using term agil...,agile day truly arrived
2,3,many exciting new digital technologies have ev...,riding the cognitive wave,"['a', 'we', 'e', 'an', 'on', 'at', 'is', 'do',...",[],many exciting new digital technology evolved r...,riding cognitive wave
3,4,we have all seen some eye bulging numbers in r...,using iot data to understand how your products...,"['hp', 'a', 'we', 'an', 'on', 'at', 'is', 's',...",['to'],seen eye bulging number recent year internet t...,using iot data understand product perform
4,5,you probably need little evidence to believe t...,the view inside our take on the customer exper...,"['a', 'we', 'e', 'an', 'i', 'on', 'at', 'is', ...",['on'],probably need little evidence believe experien...,view inside take customer experience revolution


In [ ]:
df.tail()

,asset_id,asset_description,asset_title,custom_stop_desc,custom_stop_title,asset_description_cleaned,asset_title_cleaned
77,80,this report is based on tcs larger study on di...,tcs cmo study how retailers are digitally pers...,"['3a', 'a', 'we', 'on', 'at', 'is', 's', 'do',...",['to'],report based tc larger study digital marketing...,tc cmo study retailer digitally personalizing ...
78,81,this report on marketers in the travel transpo...,tcs cmo study how travel transportation and ho...,"['a', 'we', 'e', 'on', 'is', 's', 'do', 'go', ...",['to'],report marketer travel transportation hospital...,tc cmo study travel transportation hospitality...
79,83,chief marketing officers have one of the riski...,tcs cmo study stage conversion personalizing c...,"['a', 'on', 'at', 'is', 's', 'do', '■', 'vs', ...",['to'],chief marketing officer one riskiest role suit...,tc cmo study stage conversion personalizing co...
80,84,the rise of the chief medical officer typicall...,employee wellness analysis creating data drive...,"['a', 'an', 'on', 'at', 'is', 's', 'up', 'by',...",[],rise chief medical officer typically business ...,employee wellness analysis creating data drive...
81,85,abstract as nations around the world deal with...,why the emotional wellbeing of your employees ...,"['a', 'we', 'an', 'on', 'is', 's', 'up', 'by',...","['a', 'of', 'be']",abstract nation around world deal managing cov...,emotional wellbeing employee top priority covid


# TF IDF CONTENT BASED RECOMENDATION

## TF-IDF Vectorization

### Will be using the asset_description_cleaned and asset_title_cleaned column as input for recommendation and asset_id to track the file


In [ ]:
#Define a TF-IDF Vectorizer Object. Not be using Remove all english stop words such as 'the', 'a'its already done
tfidf_desc = TfidfVectorizer()

In [ ]:
#Define a TF-IDF Vectorizer Object. Not be using Remove all english stop words such as 'the', 'a'its already done
tfidf_title = TfidfVectorizer()

In [ ]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix_desc = tfidf_desc.fit_transform(df['asset_description_cleaned'])

In [ ]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix_title = tfidf_title.fit_transform(df['asset_title_cleaned'])

In [ ]:
#Output the shape of tfidf_matrix
tfidf_matrix_desc.shape

(82, 9093)

In [ ]:
tfidf_matrix_title.shape

(82, 316)

## Similarity Matrix Calculation

### We will use cosine similarity and create two of them one for description and another for title


In [ ]:
# Compute the cosine similarity matrix
cosine_sim_desc = cosine_similarity(tfidf_matrix_desc, tfidf_matrix_desc)

In [ ]:
# Compute the cosine similarity matrix
cosine_sim_title = cosine_similarity(tfidf_matrix_title, tfidf_matrix_title)

In [ ]:
cosine_sim_desc.shape

(82, 82)

In [ ]:
cosine_sim_title.shape

(82, 82)

In [ ]:
cosine_sim_desc

array([[1.        , 0.24888038, 0.2057676 , ..., 0.14994396, 0.08362866,
        0.09439661],
       [0.24888038, 1.        , 0.11564058, ..., 0.06273883, 0.05448122,
        0.06467522],
       [0.2057676 , 0.11564058, 1.        , ..., 0.10790048, 0.08882692,
        0.08750221],
       ...,
       [0.14994396, 0.06273883, 0.10790048, ..., 1.        , 0.09906602,
        0.08461292],
       [0.08362866, 0.05448122, 0.08882692, ..., 0.09906602, 1.        ,
        0.35236438],
       [0.09439661, 0.06467522, 0.08750221, ..., 0.08461292, 0.35236438,
        1.        ]])

In [ ]:
cosine_sim_title

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.11131104],
       [0.        , 0.        , 0.        , ..., 0.        , 0.11131104,
        1.        ]])

### We will implement weightage factors to derive single similarity matrix

In [ ]:
d = 1
t = 0.5

In [ ]:
((d * cosine_sim_desc) + (t * cosine_sim_title))/(d+t)

array([[1.        , 0.16592025, 0.1371784 , ..., 0.09996264, 0.05575244,
        0.06293107],
       [0.16592025, 1.        , 0.07709372, ..., 0.04182589, 0.03632081,
        0.04311681],
       [0.1371784 , 0.07709372, 1.        , ..., 0.07193366, 0.05921795,
        0.05833481],
       ...,
       [0.09996264, 0.04182589, 0.07193366, ..., 1.        , 0.06604401,
        0.05640861],
       [0.05575244, 0.03632081, 0.05921795, ..., 0.06604401, 1.        ,
        0.27201326],
       [0.06293107, 0.04311681, 0.05833481, ..., 0.05640861, 0.27201326,
        1.        ]])

## Implement Recommendation Function

In [ ]:
# Function that takes in asset title as input and outputs most similar asset
def get_recommendations(title,df,n,sim_desc=cosine_sim_desc,sim_title=cosine_sim_title,d=1,t=0):
    
    title = title.lower()
    
    #copying ASSET CSV DATAFRAME FILE
    asset = df
    
    input_asset_id = asset["asset_id"][asset['asset_title'] == title]
    
    # Reset index of main DataFrame and construct reverse mapping as before
    asset = asset.reset_index(drop=True)
    indices = pd.Series(asset.index, index=asset['asset_title'])

    # Implementing weightage calculation
    print("desc weight factor",d)
    print("title weight factor ",t)
    
    cosine_sim = ((d * sim_desc) + (t * sim_title))/(d+t) 

    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the article based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar article
    sim_scores = sim_scores[1:n+1]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]
    sim_percentage = ["{:.2f}".format(i[1]*100) for i in sim_scores]
    
    recommendation  = {}
    for i in range(len(article_indices)):
        recommendation[str(i+1)+" recommendation"] =  "Asset_ID:"+str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage:"+str(sim_percentage[i])+"  Asset Title:" + str(asset['asset_title'][article_indices[i]])
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage : "+str(sim_percentage[i])+ "% asset_title " + str(asset['asset_title'][article_indices[i]])+"\n")
    return recommendation

NameError: ignored

In [ ]:
article_name = input("Enter the article title user visited last : \n")

Enter the article title user visited last : 
riding the cognitive wave


In [ ]:
get_recommendations(article_name,df,3)

desc weight factor 1
title weight factor  0
1th recommendation  asset_id 38 Recommending Percentage : 33.26% asset_title where to turn ai and automation loose in your company

2th recommendation  asset_id 59 Recommending Percentage : 29.42% asset_title taking a machine first approach to digital transformation

3th recommendation  asset_id 55 Recommending Percentage : 28.93% asset_title digital data s greatest opportunities



{'1 recommendation': 'Asset_ID:38 Recommending Percentage:33.26  Asset Title:where to turn ai and automation loose in your company',
 '2 recommendation': 'Asset_ID:59 Recommending Percentage:29.42  Asset Title:taking a machine first approach to digital transformation',
 '3 recommendation': 'Asset_ID:55 Recommending Percentage:28.93  Asset Title:digital data s greatest opportunities'}

## Pickling For Later Use

In [ ]:
df.to_pickle("/content/drive/MyDrive/Content_Recommendation/recommendation_v3.pkl")

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/cosine_sim_desc.pkl', 'wb') as csd:
   pickle.dump(cosine_sim_desc, csd)

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/cosine_sim_title.pkl', 'wb') as cst:
   pickle.dump(cosine_sim_title, cst)

## Test File Content

In [ ]:
import pandas as pd
import pickle

In [ ]:
# Function that takes in asset title as input and outputs most similar asset
def get_recommendations_test(title,n,d=1,t=0):
    
    title = title.lower()
    
    ##READING ASSET CSV DATAFRAME PICKLE FILE
    asset = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/recommendation_v3.pkl")
    
    input_asset_id = asset["asset_id"][asset['asset_title'] == title]
    
    # Reset index of main DataFrame and construct reverse mapping as before
    asset = asset.reset_index(drop=True)
    indices = pd.Series(asset.index, index=asset['asset_title'])

    # Reading Similarity Matrix
    
    sim_desc = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/cosine_sim_desc.pkl", "rb" ) )
    sim_title = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/cosine_sim_title.pkl", "rb" ) )

    # Implementing weightage calculation
    print("desc weight factor",d)
    print("title weight factor ",t)
    
    cosine_sim = ((d * sim_desc) + (t * sim_title))/(d+t) 

    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the article based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar article
    sim_scores = sim_scores[1:n+1]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]
    sim_percentage = ["{:.2f}".format(i[1]*100) for i in sim_scores]
    
    recommendation  = {}
    for i in range(len(article_indices)):
        recommendation[str(i+1)+" recommendation"] =  "Asset_ID:"+str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage:"+str(sim_percentage[i])+"  Asset Title:" + str(asset['asset_title'][article_indices[i]])
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage : "+str(sim_percentage[i])+ "% asset_title " + str(asset['asset_title'][article_indices[i]])+"\n")
    return recommendation

In [ ]:
title_input = input("Enter the article title user visited last : \n")

Enter the article title user visited last : 
riding the cognitive wave


In [ ]:
get_recommendations_test(title_input,3)

desc weight factor 1
title weight factor  0
1th recommendation  asset_id 38 Recommending Percentage : 33.26% asset_title where to turn ai and automation loose in your company

2th recommendation  asset_id 59 Recommending Percentage : 29.42% asset_title taking a machine first approach to digital transformation

3th recommendation  asset_id 55 Recommending Percentage : 28.93% asset_title digital data s greatest opportunities



{'1 recommendation': 'Asset_ID:38 Recommending Percentage:33.26  Asset Title:where to turn ai and automation loose in your company',
 '2 recommendation': 'Asset_ID:59 Recommending Percentage:29.42  Asset Title:taking a machine first approach to digital transformation',
 '3 recommendation': 'Asset_ID:55 Recommending Percentage:28.93  Asset Title:digital data s greatest opportunities'}

# TF-IDF User Preference/Query/Input Based Recommendation

## Query Input

### Backend list to append similar words to query

In [ ]:
preference  = [
 ['Machine Learning','Classification','Regression','ML','Artificial Intelligence','Deep Learning'],
 ['Neural Models' , 'Neural Network','CNN', 'Classification','Artificial Intelligence','Deep Learning','Convolution','AI','NLP'],
 ['Clustering & Word/Sentence Embeddings','Embeddings','Glove','Pretrained Embeddings','Clustering','Clusters'],
 ['Topic Models','Topic','Clusters','Keywords'],
 ['Language Modelling','Language Modeling','Natural Language','Language','Generation','NLP','Processing'],
 ['Segmentation','Detection','Partition','Tagging','Tags','Parsing','Parser','Object'],
 ['Sequential Labeling & Information Extraction','Labeling','Sequence','Label'],
 ['Machine Translation & Transliteration, Sequence-to-Sequence Models','Translation','Sequence','RNN','Recurrent','Neural', 'Network'],
 ['Automatic Text Summarization','Text','Summary'],
 ['Question Answering and Machine Comprehension','Question','Answering','Sequence','Model'],
 ['Generation, Reinforcement Learning','Machine Learning','Unsupervised']

]

In [ ]:
for i in range(len(preference)):
    for j in range(len(preference[i])):
        preference[i][j]= preference[i][j].lower()


In [ ]:
print(preference)

[['machine learning', 'classification', 'regression', 'ml', 'artificial intelligence', 'deep learning'], ['neural models', 'neural network', 'cnn', 'classification', 'artificial intelligence', 'deep learning', 'convolution', 'ai', 'nlp'], ['clustering & word/sentence embeddings', 'embeddings', 'glove', 'pretrained embeddings', 'clustering', 'clusters'], ['topic models', 'topic', 'clusters', 'keywords'], ['language modelling', 'language modeling', 'natural language', 'language', 'generation', 'nlp', 'processing'], ['segmentation', 'detection', 'partition', 'tagging', 'tags', 'parsing', 'parser', 'object'], ['sequential labeling & information extraction', 'labeling', 'sequence', 'label'], ['machine translation & transliteration, sequence-to-sequence models', 'translation', 'sequence', 'rnn', 'recurrent', 'neural', 'network'], ['automatic text summarization', 'text', 'summary'], ['question answering and machine comprehension', 'question', 'answering', 'sequence', 'model'], ['generation, r

### User Query

In [ ]:
query = input("Enter the query: \n")

Enter the query: 
Deep learning


In [ ]:
n = input("Enter number of recommendations you want: \n")

Enter number of recommendations you want: 
20


## Query Processing

### Preprocess User Query Append and Lamitization

#### Import For Preprocessing

In [ ]:
import nltk

In [ ]:
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Function To Append and Preprocess

In [ ]:
def process_query(query,flag,backend_list=preference):
    # flag 1 to backend append else 0
    query = query.lower()
    if flag == 0:
        return query
    elif flag == 1:
        out_query = [query]
        for i in range(len(backend_list)):
            if query in backend_list[i]:
                for j in range(len(backend_list[i])):
                    out_query.append(backend_list[i][j])
        user_preference_list = list(set(out_query))
        #Lemmatisation
        text = []
        lem = WordNetLemmatizer()
        for word in user_preference_list:
            for w in word.split(" "):
                text.append(lem.lemmatize(w)) 
        ui_query = ' '.join(text)
        return ui_query



In [ ]:
result_query = process_query(query,1)

In [ ]:
print(result_query)

ai ml classification artificial intelligence neural network machine learning neural model cnn convolution regression deep learning nlp


## Vectorizers and Matrices Processing

### Vectorisers needed in Recommendation

In [ ]:
tfidf_desc

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
tfidf_title

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
print(tfidf_matrix_desc)
print("-"*60)
print(tfidf_matrix_title)

  (0, 5482)	0.019008363018617806
  (0, 5030)	0.024948179578776104
  (0, 123)	0.032283837908646085
  (0, 644)	0.035878669548108644
  (0, 3834)	0.0193989682727746
  (0, 996)	0.03049739088477761
  (0, 3089)	0.021185415296643077
  (0, 2802)	0.03783304921464759
  (0, 2457)	0.029037757386057333
  (0, 6979)	0.01729852729356202
  (0, 1500)	0.01491878521133658
  (0, 3643)	0.03783304921464759
  (0, 4965)	0.01563620399064157
  (0, 4169)	0.03049739088477761
  (0, 6155)	0.024948179578776104
  (0, 5413)	0.029037757386057333
  (0, 6100)	0.03049739088477761
  (0, 4612)	0.03049739088477761
  (0, 6992)	0.02418514347016792
  (0, 5135)	0.024948179578776104
  (0, 9013)	0.017612521248906124
  (0, 4401)	0.03783304921464759
  (0, 4496)	0.10443149647935605
  (0, 45)	0.06362205866510606
  (0, 5535)	0.03801672603723561
  :	:
  (81, 2655)	0.03095788145189133
  (81, 3070)	0.020283809864532237
  (81, 6332)	0.0318759961512553
  (81, 4656)	0.04526209234704548
  (81, 7789)	0.013975635494612584
  (81, 8221)	0.021880222

In [ ]:
print(tfidf_desc.get_feature_names())
print(tfidf_title.get_feature_names())

['aacsb', 'aashish', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abb', 'abhors', 'abides', 'abiding', 'ability', 'able', 'abn', 'abnormal', 'abound', 'abroadway', 'abruptly', 'absence', 'absolutely', 'absorb', 'absorbed', 'absorbing', 'abstract', 'abundance', 'abundant', 'abuzz', 'academia', 'academic', 'academy', 'accelerate', 'accelerated', 'accelerates', 'accelerating', 'acceleration', 'accelerator', 'accent', 'accentuate', 'accenture', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'accepts', 'access', 'accessed', 'accessibility', 'accessible', 'accessing', 'accident', 'acclaim', 'accommodate', 'accommodated', 'accommodation', 'accompany', 'accompanying', 'accomplish', 'accomplished', 'accomplishment', 'according', 'accordingly', 'account', 'accountability', 'accountable', 'accountant', 'accounted', 'accounting', 'accredited', 'accrued', 'accumulate', 'accumulated', 'accumulating', 'accuracy', 'accurate', 'accurately', 'accustomed', 'ace', 'achievable', 'a

In [ ]:
print(tfidf_matrix_desc.shape)
print("-"*60)
print(tfidf_matrix_title.shape)

(82, 9093)
------------------------------------------------------------
(82, 316)


### Creating User Input Vectorisers

In [ ]:
test_set = []
test_set.append(result_query)

In [ ]:
print(test_set)

['ai ml classification artificial intelligence neural network machine learning neural model cnn convolution regression deep learning nlp']


In [ ]:
tfidf_matrix_ui_desc = tfidf_desc.transform(test_set)
tfidf_matrix_ui_title = tfidf_title.transform(test_set)
print(tfidf_matrix_ui_desc.shape)
print(tfidf_matrix_ui_title.shape) 

(1, 9093)
(1, 316)


### Calculating Similarity Matrix

In [ ]:
# Compute the cosine similarity matrix
cosine_sim_ui_desc = cosine_similarity(tfidf_matrix_ui_desc, tfidf_matrix_desc).flatten()
print(cosine_sim_ui_desc.shape)
print(cosine_sim_ui_desc)


(82,)
[0.02416513 0.00901018 0.05448648 0.00970376 0.00714983 0.0075275
 0.02707341 0.01858275 0.0173683  0.01188258 0.01668841 0.00942477
 0.15673538 0.         0.02406699 0.00559456 0.00624094 0.0018804
 0.01503139 0.00304333 0.         0.00429161 0.00440149 0.00415757
 0.00252088 0.002556   0.12779081 0.03374044 0.07000135 0.0086484
 0.02170512 0.03671439 0.00687075 0.00548892 0.02233194 0.
 0.00295037 0.16104395 0.01733147 0.01771421 0.00926178 0.01990392
 0.         0.02118755 0.03071608 0.014268   0.0075244  0.
 0.02843697 0.00537679 0.00386269 0.00384418 0.0160283  0.10133129
 0.0302396  0.03328205 0.03185876 0.09059933 0.0116458  0.05354508
 0.00648332 0.02639893 0.00254157 0.0212196  0.02207436 0.00574128
 0.028536   0.04516633 0.00719104 0.007486   0.0125579  0.01377827
 0.01031501 0.007106   0.00506545 0.0056753  0.002975   0.
 0.         0.         0.         0.00842937]


In [ ]:
# Compute the cosine similarity matrix
cosine_sim_ui_title = cosine_similarity(tfidf_matrix_ui_title, tfidf_matrix_title).flatten()
print(cosine_sim_ui_title.shape)
print(cosine_sim_ui_title)


(82,)
[0.         0.         0.         0.         0.         0.
 0.28138643 0.         0.         0.         0.         0.
 0.3353479  0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.13538284 0.18267112 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.14388338 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.1133701
 0.         0.         0.         0.11534326 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


In [ ]:
d = 1
t = 0

In [ ]:
cosine_sim_ui =  ((t * cosine_sim_ui_title) + (d * cosine_sim_ui_desc))/(d+t)

In [ ]:
cosine_sim_ui

array([0.02416513, 0.00901018, 0.05448648, 0.00970376, 0.00714983,
       0.0075275 , 0.02707341, 0.01858275, 0.0173683 , 0.01188258,
       0.01668841, 0.00942477, 0.15673538, 0.        , 0.02406699,
       0.00559456, 0.00624094, 0.0018804 , 0.01503139, 0.00304333,
       0.        , 0.00429161, 0.00440149, 0.00415757, 0.00252088,
       0.002556  , 0.12779081, 0.03374044, 0.07000135, 0.0086484 ,
       0.02170512, 0.03671439, 0.00687075, 0.00548892, 0.02233194,
       0.        , 0.00295037, 0.16104395, 0.01733147, 0.01771421,
       0.00926178, 0.01990392, 0.        , 0.02118755, 0.03071608,
       0.014268  , 0.0075244 , 0.        , 0.02843697, 0.00537679,
       0.00386269, 0.00384418, 0.0160283 , 0.10133129, 0.0302396 ,
       0.03328205, 0.03185876, 0.09059933, 0.0116458 , 0.05354508,
       0.00648332, 0.02639893, 0.00254157, 0.0212196 , 0.02207436,
       0.00574128, 0.028536  , 0.04516633, 0.00719104, 0.007486  ,
       0.0125579 , 0.01377827, 0.01031501, 0.007106  , 0.00506

## Recommendation 

In [ ]:
related_docs_indices = cosine_sim_ui.argsort()[:-20-1:-1] #20 recommendations

In [ ]:
related_docs_indices

array([37, 12, 26, 53, 57, 28,  2, 59, 67, 31, 27, 55, 56, 44, 54, 66, 48,
        6, 61,  0])

In [ ]:
index_in_asset = list(related_docs_indices[0:])
print(index_in_asset)

[37, 12, 26, 53, 57, 28, 2, 59, 67, 31, 27, 55, 56, 44, 54, 66, 48, 6, 61, 0]


In [ ]:
sim_percentage = ["{:.2f}".format(cosine_sim_ui[i]*100) for i in index_in_asset]
print(sim_percentage)

['16.10', '15.67', '12.78', '10.13', '9.06', '7.00', '5.45', '5.35', '4.52', '3.67', '3.37', '3.33', '3.19', '3.07', '3.02', '2.85', '2.84', '2.71', '2.64', '2.42']


In [ ]:
# # Function that takes in asset title as input and outputs most similar asset
def get_recommendations(df,n,sim_desc = cosine_sim_ui_desc,sim_title = cosine_sim_ui_title,d=1,t=0):
    
    
    #READING ASSET CSV DATAFRAME PICKLE FILE
    asset = df
    cosine_sim = ((t * sim_title) + (d * sim_desc))/(d+t)
    #find the top n related articles, we can use argsort and some negative array slicing (most related articles have highest cosine similarity values, hence at the end of the sorted indices array)
    related_docs_indices = cosine_sim.argsort()[:-n-1:-1]
    
    index_in_asset = list(related_docs_indices[0:])
    sim_percentage = ["{:.2f}".format(cosine_sim[i]*100) for i in index_in_asset]
    print(index_in_asset)
    recommendation  = {}
    for i in range(len(index_in_asset)):
        recommendation[i+1] =  "Asset_ID:"+str(asset['asset_id'][index_in_asset[i]]) + " Recommending Percentage:"+str(sim_percentage[i])+ " %  Asset Title:" + str(asset['asset_title'][index_in_asset[i]])
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['asset_id'][index_in_asset[i]]) + " Recommending Percentage:"+str(sim_percentage[i]) + " asset_title " + str(asset['asset_title'][index_in_asset[i]])+"\n")
    return recommendation
    

In [ ]:
get_recommendations(df,20)

[37, 12, 26, 53, 57, 28, 2, 59, 67, 31, 27, 55, 56, 44, 54, 66, 48, 6, 61, 0]
1th recommendation  asset_id 38 Recommending Percentage:16.10 asset_title where to turn ai and automation loose in your company

2th recommendation  asset_id 13 Recommending Percentage:15.67 asset_title how microsoft uses machine learning to handle workloads that humans cannot

3th recommendation  asset_id 27 Recommending Percentage:12.78 asset_title cloudera the mind of the new machine

4th recommendation  asset_id 54 Recommending Percentage:10.13 asset_title the big opportunities at the junction of ai and analytics interview with tom davenport

5th recommendation  asset_id 59 Recommending Percentage:9.06 asset_title taking a machine first approach to digital transformation

6th recommendation  asset_id 29 Recommending Percentage:7.00 asset_title students for life why executives must continually update their wisdom

7th recommendation  asset_id 3 Recommending Percentage:5.45 asset_title riding the cognitive 

{1: 'Asset_ID:38 Recommending Percentage:16.10 %  Asset Title:where to turn ai and automation loose in your company',
 2: 'Asset_ID:13 Recommending Percentage:15.67 %  Asset Title:how microsoft uses machine learning to handle workloads that humans cannot',
 3: 'Asset_ID:27 Recommending Percentage:12.78 %  Asset Title:cloudera the mind of the new machine',
 4: 'Asset_ID:54 Recommending Percentage:10.13 %  Asset Title:the big opportunities at the junction of ai and analytics interview with tom davenport',
 5: 'Asset_ID:59 Recommending Percentage:9.06 %  Asset Title:taking a machine first approach to digital transformation',
 6: 'Asset_ID:29 Recommending Percentage:7.00 %  Asset Title:students for life why executives must continually update their wisdom',
 7: 'Asset_ID:3 Recommending Percentage:5.45 %  Asset Title:riding the cognitive wave',
 8: 'Asset_ID:61 Recommending Percentage:5.35 %  Asset Title:operating in an insight driven world mastering the new game of analytics',
 9: 'Asset_ID

## Pickling For later use

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/tfidf_desc.pkl', 'wb') as tfd:
   pickle.dump(tfidf_desc, tfd)

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/tfidf_title.pkl', 'wb') as tft:
   pickle.dump(tfidf_title, tft)

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/tfidf_matrix_desc.pkl', 'wb') as tmd:
   pickle.dump(tfidf_matrix_desc, tmd)

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/tfidf_matrix_title.pkl', 'wb') as tmt:
   pickle.dump(tfidf_matrix_title, tmt)

## Test File Of User Input Based Recommendation

### Import 

In [ ]:
import pickle
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Query Input And Preprocessing

In [ ]:
preference  = [
 ['Machine Learning','Classification','Regression','ML','Artificial Intelligence','Deep Learning'],
 ['Neural Models' , 'Neural Network','CNN', 'Classification','Artificial Intelligence','Deep Learning','Convolution','AI','NLP'],
 ['Clustering & Word/Sentence Embeddings','Embeddings','Glove','Pretrained Embeddings','Clustering','Clusters'],
 ['Topic Models','Topic','Clusters','Keywords'],
 ['Language Modelling','Language Modeling','Natural Language','Language','Generation','NLP','Processing'],
 ['Segmentation','Detection','Partition','Tagging','Tags','Parsing','Parser','Object'],
 ['Sequential Labeling & Information Extraction','Labeling','Sequence','Label'],
 ['Machine Translation & Transliteration, Sequence-to-Sequence Models','Translation','Sequence','RNN','Recurrent','Neural', 'Network'],
 ['Automatic Text Summarization','Text','Summary'],
 ['Question Answering and Machine Comprehension','Question','Answering','Sequence','Model'],
 ['Generation, Reinforcement Learning','Machine Learning','Unsupervised']

]

In [ ]:
for i in range(len(preference)):
    for j in range(len(preference[i])):
        preference[i][j]= preference[i][j].lower()


In [ ]:
query = input("Enter the query: \n")

Enter the query: 
Deep Learning


In [ ]:
n = input("Enter number of recommendations you want: \n")

Enter number of recommendations you want: 
5


In [ ]:
def process_query_test(query,flag,backend_list=preference):
    # flag 1 to backend append else 0
    query = query.lower()
    if flag == 0:
        return query
    elif flag == 1:
        out_query = [query]
        for i in range(len(backend_list)):
            if query in backend_list[i]:
                for j in range(len(backend_list[i])):
                    out_query.append(backend_list[i][j])
        user_preference_list = list(set(out_query))
        #Lemmatisation
        text = []
        lem = WordNetLemmatizer()
        for word in user_preference_list:
            for w in word.split(" "):
                text.append(lem.lemmatize(w)) 
        ui_query = ' '.join(text)
        return ui_query



In [ ]:
result_query = process_query_test(query,1)

In [ ]:
print(result_query)

regression deep learning neural network ml convolution machine learning classification neural model cnn nlp ai artificial intelligence


### Recommendation Fuction

In [ ]:
# # Function that takes in asset title as input and outputs most similar asset
def get_query_recommendations_test(result_query,n,d=1,t=0):
    
    
    #READING ASSET CSV DATAFRAME PICKLE FILE
    asset = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/recommendation_v3.pkl")
    
    
    #MODEL
    test_set = []
    test_set.append(result_query)       

    
    #load the content
    tfidf_desc_loaded = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/tfidf_desc.pkl", "rb" ) )
    tfidf_title_loaded = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/tfidf_title.pkl", "rb" ) )

    tfidf_matrix_desc_loaded = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/tfidf_matrix_desc.pkl", "rb" ) )
    tfidf_matrix_title_loaded = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/tfidf_matrix_title.pkl", "rb" ) )
    
    # Transform Test Query to Vectorizer

    tfidf_matrix_ui_desc = tfidf_desc_loaded.transform(test_set)
    tfidf_matrix_ui_title = tfidf_title_loaded.transform(test_set)

    # Compute the cosine similarity matrix
    cosine_sim_ui_desc = cosine_similarity(tfidf_matrix_ui_desc, tfidf_matrix_desc_loaded).flatten()
    cosine_sim_ui_title = cosine_similarity(tfidf_matrix_ui_title, tfidf_matrix_title_loaded).flatten()

    # Implementing weightage calculation
    print("desc weight factor",d)
    print("title weight factor ",t)

    cosine_sim_ui =  ((t * cosine_sim_ui_title) + (d * cosine_sim_ui_desc))/(d+t)
    
    #find the top 3 related articles, we can use argsort and some negative array slicing (most related articles have highest cosine similarity values, hence at the end of the sorted indices array)
 
    related_docs_indices = cosine_sim_ui.argsort()[:-n-1:-1]
    
    index_in_asset = list(related_docs_indices[0:])
    sim_percentage = ["{:.2f}".format(cosine_sim_ui[i]*100) for i in index_in_asset]
    print(index_in_asset)
    recommendation  = {}
    for i in range(len(index_in_asset)):
        recommendation[i+1] =  "Asset_ID:"+str(asset['asset_id'][index_in_asset[i]]) + " Recommending Percentage:"+str(sim_percentage[i])+ " %  Asset Title:" + str(asset['asset_title'][index_in_asset[i]])
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['asset_id'][index_in_asset[i]]) + " Recommending Percentage:"+str(sim_percentage[i]) + " asset_title " + str(asset['asset_title'][index_in_asset[i]])+"\n")
    return recommendation
    

In [ ]:
get_query_recommendations_test(result_query,int(n))

desc weight factor 1
title weight factor  0
[37, 12, 26, 53, 57]
1th recommendation  asset_id 38 Recommending Percentage:16.10 asset_title where to turn ai and automation loose in your company

2th recommendation  asset_id 13 Recommending Percentage:15.67 asset_title how microsoft uses machine learning to handle workloads that humans cannot

3th recommendation  asset_id 27 Recommending Percentage:12.78 asset_title cloudera the mind of the new machine

4th recommendation  asset_id 54 Recommending Percentage:10.13 asset_title the big opportunities at the junction of ai and analytics interview with tom davenport

5th recommendation  asset_id 59 Recommending Percentage:9.06 asset_title taking a machine first approach to digital transformation



{1: 'Asset_ID:38 Recommending Percentage:16.10 %  Asset Title:where to turn ai and automation loose in your company',
 2: 'Asset_ID:13 Recommending Percentage:15.67 %  Asset Title:how microsoft uses machine learning to handle workloads that humans cannot',
 3: 'Asset_ID:27 Recommending Percentage:12.78 %  Asset Title:cloudera the mind of the new machine',
 4: 'Asset_ID:54 Recommending Percentage:10.13 %  Asset Title:the big opportunities at the junction of ai and analytics interview with tom davenport',
 5: 'Asset_ID:59 Recommending Percentage:9.06 %  Asset Title:taking a machine first approach to digital transformation'}

# Word2Vec Recommendation

## Library Import and Environment Preparation

In [ ]:
import numpy as np
import nltk
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

## Similarity Matrices Creation

### For description Column

In [ ]:
sentences_desc=[]
for asset in df['asset_description_cleaned']:
    sents=tokenizer.tokenize(asset.strip())
    for sent in sents:
        sentences_desc.append(sent.split())

In [ ]:
print(len(sentences_desc))

82


In [ ]:
for row in sentences_desc:
    print(row)

['peter', 'drucker', 'legendary', 'management', 'scholar', 'consultant', 'noted', 'harvard', 'business', 'review', 'article', 'assumption', 'business', 'make', 'customer', 'behavior', 'competitor', 'market', 'constitute', 'theory', 'business', 'theory', 'company', 'develops', 'dictate', 'it', 'it', 'it', 'powerful', 'tool', 'guide', 'enterprise', 'however', 'assumption', 'longer', 'reflect', 'reality', 'decision', 'company', 'make', 'led', 'profitability', 'lead', 'it', 'astray', 'failure', 'change', 'theory', 'assumption', 'undergird', 'it', 'align', 'new', 'reality', 'drucker', 'wrote', 'root', 'business', 'crisis', 'today', 'customer', 'behavior', 'market', 'competitor', 'changed', 'continue', 'change', 'rapidly', 'business', 'theory', 'reflect', 'reality', 'long', 'many', 'company', 'especially', 'affected', 'digitization', 'everything', 'crisis', 'it', 'need', 'revise', 'theory', 'business', 'become', 'smarter', 'faster', 'lighter', 'revolution', 'customer', 'behaviorhow', 'much',

In [ ]:
len_list = [len(lst) for lst in sentences_desc] 

In [ ]:
print(len_list)

[1019, 571, 1111, 851, 805, 1779, 552, 439, 1966, 1516, 1960, 1285, 448, 947, 476, 1027, 1121, 1039, 1174, 957, 409, 1197, 1242, 1632, 1470, 1428, 447, 451, 1318, 1434, 1201, 861, 1162, 1122, 1400, 1849, 1199, 1668, 1731, 1347, 992, 1453, 1381, 1511, 1200, 785, 855, 981, 888, 1017, 1038, 1333, 1305, 1047, 2214, 3587, 2921, 2634, 2027, 3498, 1162, 3409, 1975, 1812, 1230, 1661, 1737, 1699, 1838, 1868, 1476, 1785, 1528, 1478, 611, 2059, 1124, 620, 488, 1151, 932, 582]


In [ ]:
max_desc = max(len_list)

In [ ]:
max_desc

3587

In [ ]:
w2v_desc_model=Word2Vec(sentences_desc, size=max_desc, window=10, min_count=1) 

In [ ]:
w2v_desc_model.train(sentences_desc,total_examples=len(df['asset_description_cleaned']),epochs=35)

(3521769, 3867605)

In [ ]:
#Lemmatisation
lem = WordNetLemmatizer()

In [ ]:
lem.lemmatize("machines")

'machine'

In [ ]:
w2v_desc_model.wv.most_similar('machine')

[('learning', 0.8299843668937683),
 ('algorithm', 0.8046329021453857),
 ('washing', 0.7707222104072571),
 ('typing', 0.7691044807434082),
 ('managementone', 0.7688929438591003),
 ('nixdorf', 0.748212456703186),
 ('olson', 0.7371243834495544),
 ('invoice', 0.7347898483276367),
 ('observational', 0.7276601791381836),
 ('physically', 0.7261406183242798)]

In [ ]:
w2v_desc_model.wv.similarity('machine','learning')

0.8299844

### For Title column

In [ ]:
sentences_title=[]
for asset in df['asset_title_cleaned']:
    sents=tokenizer.tokenize(asset.strip())
    for sent in sents:
        sentences_title.append(sent.split())

In [ ]:
print(len(sentences_title))

82


In [ ]:
for row in sentences_title:
    print(row)

['ascent', 'smarter', 'faster', 'lighter', 'enterprise']
['agile', 'day', 'truly', 'arrived']
['riding', 'cognitive', 'wave']
['using', 'iot', 'data', 'understand', 'product', 'perform']
['view', 'inside', 'take', 'customer', 'experience', 'revolution']
['smarter', 'decision', 'smart', 'company', 'get', 'value', 'analytics']
['case', 'study', 'artificial', 'intelligence', 'big', 'part', 'future']
['banking', 'financial', 'service', 'analytics', 'competitive', 'advantage']
['keep', 'customer', 'journey', 'passing', 'sell', 'date']
['accelerated', 'change', 'getting', 'faster', 'implementing', 'enterprise', 'system']
['winning', 'today', 'talent', 'race', 'identify', 'develop', 'retain', 'talent', 'better', 'rival']
['forget', 'everything', 'know', 'brick', 'click']
['microsoft', 'us', 'machine', 'learning', 'handle', 'workload', 'human', 'cannot']
['cx', 'begin', 'tweak', 'disruption']
['retail', 'understanding', 'customer', 'performance', 'faster']
['seven', 'deadly', 'myth', 'today', 

In [ ]:
len_list_title = [len(lst) for lst in sentences_title] 

In [ ]:
max_title = max(len_list_title)

In [ ]:
max_title

13

In [ ]:
w2v_title_model=Word2Vec(sentences_title, size=max_title, window=5, min_count=1) 

In [ ]:
w2v_title_model.train(sentences_title,total_examples=len(df['asset_title_cleaned']),epochs=35)

(13257, 17815)

In [ ]:
lem.lemmatize("engineering")

'engineering'

In [ ]:
w2v_title_model.wv.most_similar('engineering')

[('immense', 0.7459281086921692),
 ('davenport', 0.7089252471923828),
 ('banking', 0.6667447686195374),
 ('wave', 0.6647173166275024),
 ('identify', 0.6243863701820374),
 ('edge', 0.6225241422653198),
 ('service', 0.6097705960273743),
 ('deadly', 0.6065347194671631),
 ('intelligence', 0.603742778301239),
 ('analytics', 0.5987162590026855)]

#### Doc vector creation

##### For DESC

In [ ]:
def make_doc_vectors_desc(asset):
    doc_vec=np.zeros(3587,dtype='float64') # initialize with zeros.
    words=asset.split()
    for word in words: # iterate over all words.
        if word in w2v_desc_model.wv.vocab:
            word_vec = w2v_desc_model.wv.get_vector(word) # if word in vocab add its word vector to our doc vector
            doc_vec=doc_vec + word_vec
    return doc_vec/len(words)  # returning the averaged word vector ie the 'doc vector' for us

In [ ]:
doc_vectors_desc = []
for asset in df['asset_description_cleaned']:
    doc_vectors_desc.append(make_doc_vectors_desc(asset))
doc_vectors_desc=np.array(doc_vectors_desc)

In [ ]:
doc_vectors_desc

array([[ 4.35445017e-02,  4.43257640e-02,  1.50139565e-01, ...,
        -7.79154244e-04,  1.69254808e-02, -5.09118995e-02],
       [-1.92939395e-02, -5.56727760e-02, -1.31805239e-02, ...,
        -6.02801358e-03, -3.53903543e-02, -4.53408193e-02],
       [ 2.60585541e-02,  7.97876909e-02,  1.90278571e-01, ...,
         1.75003185e-02, -7.48079509e-03, -6.93060537e-02],
       ...,
       [ 1.42470889e-01,  1.93054725e-03,  5.03202410e-01, ...,
        -1.05023326e-02,  4.39909985e-02, -3.94937899e-02],
       [ 6.50515736e-03, -3.93268125e-02,  1.90145478e-01, ...,
        -7.13090974e-03,  3.18060748e-02,  3.29948115e-02],
       [ 1.94368308e-02, -4.10704415e-02,  1.74009185e-01, ...,
         7.84153838e-04, -7.32531693e-03,  4.77659302e-04]])

In [ ]:
doc_vectors_desc.shape

(82, 3587)

##### For TITLE

In [ ]:
def make_doc_vectors_title(asset):
    doc_vec=np.zeros(13,dtype='float64') # initialize with zeros.
    words=asset.split()
    for word in words: # iterate over all words.
        if word in w2v_title_model.wv.vocab:
            word_vec = w2v_title_model.wv.get_vector(word) # if word in vocab add its word vector to our doc vector
            doc_vec=doc_vec + word_vec
    return doc_vec/len(words)  # returning the averaged word vector ie the 'doc vector' for us

In [ ]:
doc_vectors_title = []
for asset in df['asset_title_cleaned']:
    doc_vectors_title.append(make_doc_vectors_title(asset))
doc_vectors_title=np.array(doc_vectors_title)

In [ ]:
doc_vectors_title

array([[ 0.01751998, -0.00771606,  0.01024189, ..., -0.00605063,
         0.0248749 , -0.00257159],
       [ 0.02319843, -0.02330597, -0.0039553 , ..., -0.0129207 ,
         0.00804607,  0.00479961],
       [ 0.01539538, -0.03229834,  0.02098492, ..., -0.04015631,
         0.01467216,  0.00733395],
       ...,
       [ 0.03168688, -0.03421047,  0.01133739, ..., -0.03330055,
         0.01392137,  0.00501583],
       [ 0.03836257, -0.0287282 ,  0.00527533, ..., -0.03035368,
         0.02965296, -0.00027853],
       [-0.0064157 , -0.00843496,  0.01609344, ..., -0.03157389,
         0.014542  , -0.01380835]])

In [ ]:
doc_vectors_title.shape

(82, 13)

### Similarity Matrix Creation

In [ ]:
cosine_sim_desc_w2v = cosine_similarity(doc_vectors_desc, doc_vectors_desc)

In [ ]:
cosine_sim_title_w2v = cosine_similarity(doc_vectors_title, doc_vectors_title)

In [ ]:
cosine_sim_desc_w2v

array([[1.        , 0.65730222, 0.77161746, ..., 0.53848491, 0.23716043,
        0.48792779],
       [0.65730222, 1.        , 0.42727716, ..., 0.25216207, 0.202013  ,
        0.47514787],
       [0.77161746, 0.42727716, 1.        , ..., 0.36127273, 0.44431613,
        0.56930862],
       ...,
       [0.53848491, 0.25216207, 0.36127273, ..., 1.        , 0.3803217 ,
        0.4804381 ],
       [0.23716043, 0.202013  , 0.44431613, ..., 0.3803217 , 1.        ,
        0.86543644],
       [0.48792779, 0.47514787, 0.56930862, ..., 0.4804381 , 0.86543644,
        1.        ]])

In [ ]:
cosine_sim_title_w2v

array([[1.        , 0.23631859, 0.72537256, ..., 0.58897892, 0.77798443,
        0.58116863],
       [0.23631859, 1.        , 0.41202883, ..., 0.77855669, 0.68631861,
        0.24663272],
       [0.72537256, 0.41202883, 1.        , ..., 0.73559899, 0.7641138 ,
        0.76770353],
       ...,
       [0.58897892, 0.77855669, 0.73559899, ..., 1.        , 0.84402547,
        0.57218272],
       [0.77798443, 0.68631861, 0.7641138 , ..., 0.84402547, 1.        ,
        0.60336454],
       [0.58116863, 0.24663272, 0.76770353, ..., 0.57218272, 0.60336454,
        1.        ]])

In [ ]:
cosine_sim_desc_w2v.shape,cosine_sim_title_w2v.shape

((82, 82), (82, 82))

## Implementing Recommedation Function

In [ ]:
# Function that takes in asset title as input and outputs most similar asset
def get_recommendations_w2v(title,df,n,sim_desc=cosine_sim_desc_w2v,sim_title=cosine_sim_title_w2v,d=1,t=0):
    
    title = title.lower()
    
    #copying ASSET CSV DATAFRAME FILE
    asset = df
    
    input_asset_id = asset["asset_id"][asset['asset_title'] == title]
    
    # Reset index of main DataFrame and construct reverse mapping as before
    asset = asset.reset_index(drop=True)
    indices = pd.Series(asset.index, index=asset['asset_title'])

    # Implementing weightage calculation
    print("desc weight factor",d)
    print("title weight factor ",t)
    
    cosine_sim = ((d * sim_desc) + (t * sim_title))/(d+t) 

    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the article based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar article
    sim_scores = sim_scores[1:n+1]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]
    print(article_indices)
    sim_percentage = ["{:.2f}".format(i[1]*100) for i in sim_scores]
    
    recommendation  = {}
    for i in range(len(article_indices)):
        recommendation[str(i+1)+" recommendation"] =  "Asset_ID:"+str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage:"+str(sim_percentage[i])+"  Asset Title:" + str(asset['asset_title'][article_indices[i]])
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage : "+str(sim_percentage[i])+ "% asset_title " + str(asset['asset_title'][article_indices[i]])+"\n")
    return recommendation

In [ ]:
article_name = input("Enter the article title user visited last : \n")

Enter the article title user visited last : 
riding the cognitive wave


In [ ]:
get_recommendations_w2v(article_name,df,3)

desc weight factor 1
title weight factor  0
[26, 12, 53]
1th recommendation  asset_id 27 Recommending Percentage : 94.37% asset_title cloudera the mind of the new machine

2th recommendation  asset_id 13 Recommending Percentage : 88.41% asset_title how microsoft uses machine learning to handle workloads that humans cannot

3th recommendation  asset_id 54 Recommending Percentage : 87.67% asset_title the big opportunities at the junction of ai and analytics interview with tom davenport



{'1 recommendation': 'Asset_ID:27 Recommending Percentage:94.37  Asset Title:cloudera the mind of the new machine',
 '2 recommendation': 'Asset_ID:13 Recommending Percentage:88.41  Asset Title:how microsoft uses machine learning to handle workloads that humans cannot',
 '3 recommendation': 'Asset_ID:54 Recommending Percentage:87.67  Asset Title:the big opportunities at the junction of ai and analytics interview with tom davenport'}

## Pickling For Later Use

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/cosine_sim_desc_w2v.pkl', 'wb') as cdw:
   pickle.dump(cosine_sim_desc_w2v, cdw)

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/cosine_sim_title_w2v.pkl', 'wb') as ctw:
   pickle.dump(cosine_sim_title_w2v, ctw)

## Test File Word2Vec Last Read

In [ ]:
import pandas as pd
import pickle

In [ ]:
# Function that takes in asset title as input and outputs most similar asset
def get_recommendations_test_w2v(title,n,d=1,t=0):
    
    title = title.lower()
    
    ##READING ASSET CSV DATAFRAME PICKLE FILE
    asset = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/recommendation_v3.pkl")
    
    input_asset_id = asset["asset_id"][asset['asset_title'] == title]
    
    # Reset index of main DataFrame and construct reverse mapping as before
    asset = asset.reset_index(drop=True)
    indices = pd.Series(asset.index, index=asset['asset_title'])

    # Reading Similarity Matrix
    
    sim_desc = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/cosine_sim_desc_w2v.pkl", "rb" ) )
    sim_title = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/cosine_sim_title_w2v.pkl", "rb" ) )

    # Implementing weightage calculation
    print("desc weight factor",d)
    print("title weight factor ",t)
    
    cosine_sim = ((d * sim_desc) + (t * sim_title))/(d+t) 

    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the article based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar article
    sim_scores = sim_scores[1:n+1]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]
    sim_percentage = ["{:.2f}".format(i[1]*100) for i in sim_scores]
    
    recommendation  = {}
    for i in range(len(article_indices)):
        recommendation[str(i+1)+" recommendation"] =  "Asset_ID:"+str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage:"+str(sim_percentage[i])+"  Asset Title:" + str(asset['asset_title'][article_indices[i]])
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage : "+str(sim_percentage[i])+ "% asset_title " + str(asset['asset_title'][article_indices[i]])+"\n")
    return recommendation

In [ ]:
article_input = input("Enter the article title user visited last : \n")

Enter the article title user visited last : 
riding the cognitive wave


In [ ]:
get_recommendations_test_w2v(article_input,3)

desc weight factor 1
title weight factor  0
1th recommendation  asset_id 27 Recommending Percentage : 94.37% asset_title cloudera the mind of the new machine

2th recommendation  asset_id 13 Recommending Percentage : 88.41% asset_title how microsoft uses machine learning to handle workloads that humans cannot

3th recommendation  asset_id 54 Recommending Percentage : 87.67% asset_title the big opportunities at the junction of ai and analytics interview with tom davenport



{'1 recommendation': 'Asset_ID:27 Recommending Percentage:94.37  Asset Title:cloudera the mind of the new machine',
 '2 recommendation': 'Asset_ID:13 Recommending Percentage:88.41  Asset Title:how microsoft uses machine learning to handle workloads that humans cannot',
 '3 recommendation': 'Asset_ID:54 Recommending Percentage:87.67  Asset Title:the big opportunities at the junction of ai and analytics interview with tom davenport'}

# Glove Based Recommendation

## Library Import and glove file read

In [ ]:
import numpy as np
from gensim.scripts.glove2word2vec import glove2word2vec

from gensim.models import KeyedVectors

In [ ]:

embeddings_file = '/content/drive/MyDrive/WordPrediction/glove.6B.100d.txt'

In [ ]:
word2vec_output_file = '{0}.word2vec'.format(embeddings_file)
glove2word2vec(embeddings_file, word2vec_output_file)

(400000, 100)

In [ ]:
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [ ]:
glove_model.most_similar('artificial')

[('neural', 0.597928524017334),
 ('natural', 0.5964989066123962),
 ('synthetic', 0.5935590863227844),
 ('man-made', 0.5844878554344177),
 ('embryos', 0.55003422498703),
 ('water', 0.5468946099281311),
 ('tissue', 0.5433562397956848),
 ('effects', 0.5430183410644531),
 ('therapeutic', 0.5396246910095215),
 ('soft', 0.537155032157898)]

## Similarity Matrices Preparation

In [ ]:
def make_doc_vectors_glove(asset):
    doc_vec=np.zeros(100,dtype='float64') # initialize with zeros.
    words=asset.split()
    for word in words: # iterate over all words.
        if word in glove_model.wv.vocab:
            word_vec = glove_model.wv.get_vector(word) # if word in vocab add its word vector to our doc vector
            doc_vec=doc_vec + word_vec
    return doc_vec/len(words)  # returning the averaged word vector ie the 'doc vector' for us.

In [ ]:
doc_vectors_desc_g = []
for asset in df['asset_description_cleaned']:
    doc_vectors_desc_g.append(make_doc_vectors_glove(asset))
doc_vectors_desc_g=np.array(doc_vectors_desc_g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
doc_vectors_title_g = []
for asset in df['asset_title_cleaned']:
    doc_vectors_title_g.append(make_doc_vectors_glove(asset))
doc_vectors_title_g=np.array(doc_vectors_title_g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
print(doc_vectors_desc_g.shape)
print(doc_vectors_desc_g)

(82, 100)
[[-0.057966    0.09718962  0.1806827  ... -0.27881091  0.53620011
   0.2131804 ]
 [-0.08017313  0.15266759  0.06861094 ... -0.30141604  0.54393198
   0.22200815]
 [-0.0857051   0.18588429  0.2811531  ... -0.36621144  0.51698148
   0.15483374]
 ...
 [-0.05821331  0.12213775  0.12722027 ... -0.26198699  0.53148487
   0.23120707]
 [-0.07850152  0.10653059  0.17155924 ... -0.33399949  0.41943311
   0.05160826]
 [-0.06123175  0.15602914  0.10133665 ... -0.34050084  0.49499352
   0.18762799]]


In [ ]:
print(doc_vectors_title_g.shape)
print(doc_vectors_title_g)

(82, 100)
[[-0.0683918   0.18726601  0.31035021 ... -0.1400148   0.574426
   0.0436548 ]
 [-0.2910875   0.17151     0.02836    ... -0.28957     0.03186774
   0.334716  ]
 [-0.02737567 -0.03339999 -0.19509333 ... -0.67366245  0.89379001
  -0.24848334]
 ...
 [-0.078844    0.153997    0.1228407  ... -0.1371904   0.396156
   0.2189514 ]
 [-0.07550518  0.14389334  0.12824136 ... -0.34604627  0.35517137
   0.00751218]
 [ 0.06829333  0.05365433  0.254772   ... -0.32611067  0.45096001
   0.160948  ]]


In [ ]:
cosine_sim_desc_g = cosine_similarity(doc_vectors_desc_g, doc_vectors_desc_g)

In [ ]:
cosine_sim_title_g = cosine_similarity(doc_vectors_title_g, doc_vectors_title_g)

In [ ]:
print(cosine_sim_desc_g.shape)
print(cosine_sim_desc_g)

(82, 82)
[[1.         0.98673025 0.98306222 ... 0.98535846 0.94671949 0.96196904]
 [0.98673025 1.         0.97676719 ... 0.97336663 0.93186307 0.96207897]
 [0.98306222 0.97676719 1.         ... 0.97493204 0.95096038 0.97139496]
 ...
 [0.98535846 0.97336663 0.97493204 ... 1.         0.95324948 0.96582595]
 [0.94671949 0.93186307 0.95096038 ... 0.95324948 1.         0.97182363]
 [0.96196904 0.96207897 0.97139496 ... 0.96582595 0.97182363 1.        ]]


In [ ]:
print(cosine_sim_title_g.shape)
print(cosine_sim_title_g)

(82, 82)
[[1.         0.59249188 0.43444947 ... 0.61543521 0.58088675 0.41684744]
 [0.59249188 1.         0.44526807 ... 0.59749065 0.62173925 0.55143234]
 [0.43444947 0.44526807 1.         ... 0.50665619 0.58216182 0.43671661]
 ...
 [0.61543521 0.59749065 0.50665619 ... 1.         0.81096115 0.68384938]
 [0.58088675 0.62173925 0.58216182 ... 0.81096115 1.         0.79381787]
 [0.41684744 0.55143234 0.43671661 ... 0.68384938 0.79381787 1.        ]]


## Recommendation Function

In [ ]:
# Function that takes in asset title as input and outputs most similar asset
def get_recommendations_glove(title,df,n,sim_desc=cosine_sim_desc_g,sim_title=cosine_sim_title_g,d=1,t=0):
    
    title = title.lower()
    
    #copying ASSET CSV DATAFRAME FILE
    asset = df
    
    input_asset_id = asset["asset_id"][asset['asset_title'] == title]
    
    # Reset index of main DataFrame and construct reverse mapping as before
    asset = asset.reset_index(drop=True)
    indices = pd.Series(asset.index, index=asset['asset_title'])

    # Implementing weightage calculation
    print("desc weight factor",d)
    print("title weight factor ",t)
    
    cosine_sim = ((d * sim_desc) + (t * sim_title))/(d+t) 

    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the article based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar article
    sim_scores = sim_scores[1:n+1]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]
    print(article_indices)
    sim_percentage = ["{:.2f}".format(i[1]*100) for i in sim_scores]
    
    recommendation  = {}
    for i in range(len(article_indices)):
        recommendation[str(i+1)+" recommendation"] =  "Asset_ID:"+str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage:"+str(sim_percentage[i])+"  Asset Title:" + str(asset['asset_title'][article_indices[i]])
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage : "+str(sim_percentage[i])+ "% asset_title " + str(asset['asset_title'][article_indices[i]])+"\n")
    return recommendation

In [ ]:
article_name = input("Enter the article title user visited last : \n")


Enter the article title user visited last : 
riding the cognitive wave


In [ ]:
get_recommendations_glove(article_name,df,3)

desc weight factor 1
title weight factor  0
[26, 46, 53]
1th recommendation  asset_id 27 Recommending Percentage : 99.27% asset_title cloudera the mind of the new machine

2th recommendation  asset_id 47 Recommending Percentage : 99.08% asset_title analytics a still emerging business frontier interview with prabhudev konana

3th recommendation  asset_id 54 Recommending Percentage : 98.79% asset_title the big opportunities at the junction of ai and analytics interview with tom davenport



{'1 recommendation': 'Asset_ID:27 Recommending Percentage:99.27  Asset Title:cloudera the mind of the new machine',
 '2 recommendation': 'Asset_ID:47 Recommending Percentage:99.08  Asset Title:analytics a still emerging business frontier interview with prabhudev konana',
 '3 recommendation': 'Asset_ID:54 Recommending Percentage:98.79  Asset Title:the big opportunities at the junction of ai and analytics interview with tom davenport'}

## Pickling For Future Use

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/cosine_sim_desc_g.pkl', 'wb') as cdg:
   pickle.dump(cosine_sim_desc_g, cdg)

NameError: ignored

In [ ]:
with open('/content/drive/MyDrive/Content_Recommendation/cosine_sim_title_g.pkl', 'wb') as ctg:
   pickle.dump(cosine_sim_title_g, ctg)

NameError: ignored

## Test File last read glove

In [ ]:
import pandas as pd
import pickle

In [ ]:
# Function that takes in asset title as input and outputs most similar asset
def get_recommendations_test_glove(title,n,d=1,t=0):
    
    title = title.lower()
    
    ##READING ASSET CSV DATAFRAME PICKLE FILE
    asset = pd.read_pickle("/content/drive/MyDrive/Content_Recommendation/recommendation_v3.pkl")
    
    input_asset_id = asset["asset_id"][asset['asset_title'] == title]
    
    # Reset index of main DataFrame and construct reverse mapping as before
    asset = asset.reset_index(drop=True)
    indices = pd.Series(asset.index, index=asset['asset_title'])

    # Reading Similarity Matrix
    
    sim_desc = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/cosine_sim_desc_g.pkl", "rb" ) )
    sim_title = pickle.load(open("/content/drive/MyDrive/Content_Recommendation/cosine_sim_title_g.pkl", "rb" ) )

    # Implementing weightage calculation
    print("desc weight factor",d)
    print("title weight factor ",t)
    
    cosine_sim = ((d * sim_desc) + (t * sim_title))/(d+t) 

    # Get the index of the article that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the article based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the n most similar article
    sim_scores = sim_scores[1:n+1]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]
    sim_percentage = ["{:.2f}".format(i[1]*100) for i in sim_scores]
    
    recommendation  = {}
    for i in range(len(article_indices)):
        recommendation[str(i+1)+" recommendation"] =  "Asset_ID:"+str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage:"+str(sim_percentage[i])+"  Asset Title:" + str(asset['asset_title'][article_indices[i]])
        print(str(i+1) + "th recommendation " + " asset_id " + str(asset['asset_id'][article_indices[i]]) + " Recommending Percentage : "+str(sim_percentage[i])+ "% asset_title " + str(asset['asset_title'][article_indices[i]])+"\n")
    return recommendation

In [ ]:
article_input = input("Enter the article title user visited last : \n")

Enter the article title user visited last : 
riding the cognitive wave


In [ ]:
get_recommendations_test_glove(article_input,3)

desc weight factor 1
title weight factor  0
1th recommendation  asset_id 27 Recommending Percentage : 99.27% asset_title cloudera the mind of the new machine

2th recommendation  asset_id 47 Recommending Percentage : 99.08% asset_title analytics a still emerging business frontier interview with prabhudev konana

3th recommendation  asset_id 54 Recommending Percentage : 98.79% asset_title the big opportunities at the junction of ai and analytics interview with tom davenport



{'1 recommendation': 'Asset_ID:27 Recommending Percentage:99.27  Asset Title:cloudera the mind of the new machine',
 '2 recommendation': 'Asset_ID:47 Recommending Percentage:99.08  Asset Title:analytics a still emerging business frontier interview with prabhudev konana',
 '3 recommendation': 'Asset_ID:54 Recommending Percentage:98.79  Asset Title:the big opportunities at the junction of ai and analytics interview with tom davenport'}

# Testing

In [ ]:
title_list = df['asset_title'].tolist()

In [ ]:
print(title_list)

['the ascent of the smarter faster lighter enterprise', 'agile s day has truly arrived', 'riding the cognitive wave', 'using iot data to understand how your products perform', 'the view inside our take on the customer experience revolution', 'smarter decisions how smart companies get more value from analytics', 'case study why artificial intelligence is a big part of ap s future', 'banking and financial services analytics for competitive advantage', 'how to keep your customer journey from passing its sell by date', 'accelerated change getting faster at implementing enterprise systems', 'winning today s talent race how to identify develop and retain talent better than your rivals', 'forget everything you know about bricks and clicks', 'how microsoft uses machine learning to handle workloads that humans cannot', 'when cx should begin with tweaks not disruption', 'retail understanding your customer performance faster', 'the seven deadly myths of today s b2b sell', 'personalization boundar

In [ ]:
print((title_list[2]))

riding the cognitive wave


## Calling Model

In [ ]:
print("Recommendation Using TFIDF")
get_recommendations_test(title_list[2],3)

Recommendation Using TFIDF
desc weight factor 1
title weight factor  0
1th recommendation  asset_id 38 Recommending Percentage : 33.26% asset_title where to turn ai and automation loose in your company

2th recommendation  asset_id 59 Recommending Percentage : 29.42% asset_title taking a machine first approach to digital transformation

3th recommendation  asset_id 55 Recommending Percentage : 28.93% asset_title digital data s greatest opportunities



{'1 recommendation': 'Asset_ID:38 Recommending Percentage:33.26  Asset Title:where to turn ai and automation loose in your company',
 '2 recommendation': 'Asset_ID:59 Recommending Percentage:29.42  Asset Title:taking a machine first approach to digital transformation',
 '3 recommendation': 'Asset_ID:55 Recommending Percentage:28.93  Asset Title:digital data s greatest opportunities'}

In [ ]:
print("Recommendation Using Word2Vec")
get_recommendations_test_w2v(title_list[2],3)

Recommendation Using Word2Vec
desc weight factor 1
title weight factor  0
1th recommendation  asset_id 27 Recommending Percentage : 94.37% asset_title cloudera the mind of the new machine

2th recommendation  asset_id 13 Recommending Percentage : 88.41% asset_title how microsoft uses machine learning to handle workloads that humans cannot

3th recommendation  asset_id 54 Recommending Percentage : 87.67% asset_title the big opportunities at the junction of ai and analytics interview with tom davenport



{'1 recommendation': 'Asset_ID:27 Recommending Percentage:94.37  Asset Title:cloudera the mind of the new machine',
 '2 recommendation': 'Asset_ID:13 Recommending Percentage:88.41  Asset Title:how microsoft uses machine learning to handle workloads that humans cannot',
 '3 recommendation': 'Asset_ID:54 Recommending Percentage:87.67  Asset Title:the big opportunities at the junction of ai and analytics interview with tom davenport'}

In [ ]:
print("Recommendation Using Glove")
get_recommendations_test_glove(title_list[2],3)

Recommendation Using Glove
desc weight factor 1
title weight factor  0
1th recommendation  asset_id 27 Recommending Percentage : 99.27% asset_title cloudera the mind of the new machine

2th recommendation  asset_id 47 Recommending Percentage : 99.08% asset_title analytics a still emerging business frontier interview with prabhudev konana

3th recommendation  asset_id 54 Recommending Percentage : 98.79% asset_title the big opportunities at the junction of ai and analytics interview with tom davenport



{'1 recommendation': 'Asset_ID:27 Recommending Percentage:99.27  Asset Title:cloudera the mind of the new machine',
 '2 recommendation': 'Asset_ID:47 Recommending Percentage:99.08  Asset Title:analytics a still emerging business frontier interview with prabhudev konana',
 '3 recommendation': 'Asset_ID:54 Recommending Percentage:98.79  Asset Title:the big opportunities at the junction of ai and analytics interview with tom davenport'}